<a href="https://colab.research.google.com/github/stellagerantoni/LatentCfMultivariate/blob/main/NewDatasetWithoutKDE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! git clone https://github.com/stellagerantoni/LatentCfMultivariate

Cloning into 'LatentCfMultivariate'...
remote: Enumerating objects: 134, done.
remote: Counting objects: 100% (134/134), done.
remote: Compressing objects: 100% (132/132), done.
remote: Total 134 (delta 77), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (134/134), 7.20 MiB | 8.70 MiB/s, done.
Resolving deltas: 100% (77/77), done.


In [ ]:
!pip install -q wildboar
!pip install -q scikit-learn
!pip install -q stumpy
!pip install -q fastdtw
!pip install aeon

In [ ]:
import logging
import os
import warnings
from argparse import ArgumentParser
from aeon.datasets import load_classification

from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from scipy.spatial import distance_matrix
from sklearn.metrics import balanced_accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KDTree, KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from wildboar.datasets import load_dataset
from wildboar.ensemble import ShapeletForestClassifier
from wildboar.explain.counterfactual import counterfactuals
%cd '/content/LatentCfMultivariate'
from _guided import ModifiedLatentCF
from help_functions import *
from keras_models import *

/content/LatentCfMultivariate


In [ ]:
os.environ['TF_DETERMINISTIC_OPS'] = '1'
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
RANDOM_STATE = 39

## **FUNCTIONS**

In [ ]:
def load_dataset(dataset):
  X, y = load_classification(dataset)
  if dataset == 'Heartbeat':
    pos = 'normal'
    neg = 'abnormal'
  if dataset == 'SelfRegulationSCP1':
    pos = 'positivity'
    neg = 'negativity'
  if dataset == 'Blink':
    pos = 'shortblink'
    neg = 'longblink'
    X = np.array(X)
    y = np.array(y)
  if dataset == 'FingerMovements':
    pos = 'left'
    neg = 'right'
  if dataset == 'Cricket':
    return X,y,meta_data
  if dataset == 'SpokenArabicDigits':
    return X,y,meta_data
  if dataset == 'PenDigits':
    return X,y,meta_data


  print(" Shape of X = ", X.shape)
  print(" Shape of y = ", y.shape)
  #print(" Meta data = ", meta_data)
  # Convert positive and negative labels to 1 and 0
  pos_label, neg_label = 1, 0
  if pos != pos_label:
      y[y==pos] = pos_label # convert/normalize positive label to 1
  if neg != neg_label:
      y[y==neg] = neg_label # convert negative label to 0

  y = y.astype(int)
  print(f"\n X[:1] = \n{X[:1]}")
  return X,y,pos_label, neg_label

# **ACTUALL CODE**
datasets available : 'Heartbeat', 'SelfRegulationSCP1'

In [ ]:
RANDOM_STATE = 39
X,y,pos_label,neg_label = load_dataset('FingerMovements')
X = X.transpose(0,2,1)
print(f'shape of X = {X.shape}')
print(f'shape of y = {y.shape}')
#print(f'data imformation = {data_information}')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)
print(f'shape of X train = {X_train.shape}')
print(f'shape of y train = {y_train.shape}')

 Shape of X =  (416, 28, 50)
 Shape of y =  (416,)

 X[:1] = 
[[[41.8 44.8 47.1 ... 69.8 72.6 76.1]
  [55.2 53.8 59.9 ... 17.5 28.  12.1]
  [-8.6 -3.6 14.4 ... 23.3 35.9 23.2]
  ...
  [16.9 24.5 24.5 ... 51.9 59.6 57.3]
  [42.2 35.  41.7 ... 51.5 58.5 46.9]
  [13.  26.6 52.5 ... -3.5 -3.2 -2.6]]]
shape of X = (416, 50, 28)
shape of y = (416,)
shape of X train = (332, 50, 28)
shape of y train = (332,)


In [ ]:
# Upsample the minority class
unique_classes, class_counts = np.unique(y_train, return_counts=True)
print(f'before: {class_counts}')
X_train,y_train = upsample_minority_multivariate(X_train,y_train)
unique_classes, class_counts = np.unique(y_train, return_counts=True)
print(f'after: {class_counts}')

before: [166 166]
after: [166 166]


In [ ]:
n_training,n_timesteps, n_features= X_train.shape

X, trained_scaler =  normalize_multivariate(data=X, n_timesteps=n_timesteps, n_features = n_features)
X_train_processed, trained_scaler =  normalize_multivariate(data=X_train, n_timesteps=n_timesteps, n_features = n_features)
X_test_processed, _ =  normalize_multivariate(data=X_test, n_timesteps=n_timesteps, scaler=trained_scaler, n_features = n_features)

X, padding_size = conditional_pad_multivariate(X)
X_train_processed_padded, padding_size = conditional_pad_multivariate(X_train_processed) # add extra padding zeros if n_timesteps cannot be divided by 4, required for 1dCNN autoencoder structure
X_test_processed_padded, _ = conditional_pad_multivariate(X_test_processed)

n_timesteps_padded = X_train_processed_padded.shape[1]
print(f"Data pre-processed, original #timesteps={n_timesteps}, padded #timesteps={n_timesteps_padded}.")

#check the processing (0,1) min should be min 0 and max should be max 1
print(f"\nmin value = {np.min(X_train)}, max value = {np.max(X_train)}")
print(f"min value normalized = {np.min(X_train_processed)}, max value normalized= {np.max(X_train_processed)}")

#check that padding paddes the right dimention
print(f"\nX_train.shape = {X_train.shape}" )
print(f"X_train_processed_padded.shape = {X_train_processed_padded.shape}")


Data pre-processed, original #timesteps=50, padded #timesteps=52.

min value = -132.1, max value = 205.1
min value normalized = 0.0, max value normalized= 1.0

X_train.shape = (332, 50, 28)
X_train_processed_padded.shape = (332, 52, 28)


In [ ]:
from sklearn.model_selection import train_test_split
X_train_processed_padded,X_validation, y_train, y_validation = train_test_split(X_train_processed_padded, y_train, test_size=0.2, random_state=RANDOM_STATE, stratify=y_train)


In [ ]:
print(f'X_train = {X_train_processed_padded.shape}')
print(f'X_validation = {X_validation.shape}')
print(f'X_test = {X_test_processed_padded.shape}')

y_classes = y
y_train_classes = y_train
y_validation_classes = y_validation
y_test_classes = y_test

from tensorflow.keras.utils import to_categorical
y = to_categorical(y,len(np.unique(y)))
y_train = to_categorical(y_train, len(np.unique(y_train)))
y_validation = to_categorical(y_validation, len(np.unique(y_validation)))
y_test = to_categorical(y_test, len(np.unique(y_test)))

print(f'\ny_train_classes = {y_train_classes.shape}, y_validation_classes = {y_validation_classes.shape}, y_test_classes = {y_test_classes.shape}')
print(f'y_train = {y_train.shape}, y_validation = {y_validation.shape}, y_test= {y_test.shape}')

X_train = (265, 52, 28)
X_validation = (67, 52, 28)
X_test = (84, 52, 28)

y_train_classes = (265,), y_validation_classes = (67,), y_test_classes = (84,)
y_train = (265, 2), y_validation = (67, 2), y_test= (84, 2)


In [ ]:
y_train_classes.shape

(640,)

In [ ]:
y_train.shape

(640, 2)

In [ ]:


# ## 2. LatentCF models
# reset seeds for numpy, tensorflow, python random package and python environment seed
reset_seeds()
###############################################
# ### 1dCNN classifier

cnnClassifier = Classifier(
    n_timesteps_padded, n_features, n_output=2, add_dense_layer = False
)

optimizer = keras.optimizers.Adam(lr=0.001)
cnnClassifier.compile(
    optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"]
)

# Define the early stopping criteria
early_stopping_accuracy = keras.callbacks.EarlyStopping(
    monitor="val_accuracy", patience=15, restore_best_weights=True
)
# Train the model
reset_seeds()
print("Training log for LSTM-FCN classifier:")
classifier_history = cnnClassifier.fit(
    X_train_processed_padded,
    y_train,
    epochs=150,
    batch_size=12,
    shuffle=True,
    verbose=True,
    validation_data=(X_validation, y_validation),
    callbacks=[early_stopping_accuracy],
)

y_pred = cnnClassifier.predict(X_test_processed_padded)
y_pred_classes = np.argmax(y_pred, axis=1)
acc = balanced_accuracy_score(y_true=y_test_classes, y_pred=y_pred_classes)
print(f"LSTM-FCN classifier trained, with validation accuracy {acc}.")

confusion_matrix_df = pd.DataFrame(
    confusion_matrix(y_true=y_test_classes, y_pred=y_pred_classes, labels=[1, 0]),
    index=["True:8", "True:0"],
    columns=["Pred:8", "Pred:0"],
)
print(confusion_matrix_df)


Training log for LSTM-FCN classifier:
Epoch 1/150
23/23 [==============================] - 1s 17ms/step - loss: 0.8307 - accuracy: 0.5132 - val_loss: 0.7577 - val_accuracy: 0.4925
Epoch 2/150
23/23 [==============================] - 0s 7ms/step - loss: 0.6913 - accuracy: 0.6226 - val_loss: 0.7282 - val_accuracy: 0.5224
Epoch 3/150
23/23 [==============================] - 0s 7ms/step - loss: 0.6332 - accuracy: 0.7132 - val_loss: 0.7123 - val_accuracy: 0.6567
Epoch 4/150
23/23 [==============================] - 0s 7ms/step - loss: 0.7008 - accuracy: 0.6189 - val_loss: 0.7063 - val_accuracy: 0.6418
Epoch 5/150
23/23 [==============================] - 0s 7ms/step - loss: 0.5797 - accuracy: 0.7585 - val_loss: 0.7009 - val_accuracy: 0.6567
Epoch 6/150
23/23 [==============================] - 0s 7ms/step - loss: 0.5776 - accuracy: 0.7736 - val_loss: 0.7315 - val_accuracy: 0.4925
Epoch 7/150
23/23 [==============================] - 0s 7ms/step - loss: 0.5878 - accuracy: 0.7321 - val_loss: 0.73

In [ ]:
reset_seeds()


# ### 1dCNN autoencoder
autoencoder = Autoencoder(n_timesteps_padded,n_features,32)
optimizer = keras.optimizers.Adam(lr=0.0005)
autoencoder.compile(optimizer=optimizer, loss="mse")

# Define the early stopping criteria
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=5, restore_best_weights=True)
# Train the model
reset_seeds()
print("Training log for 1dCNN autoencoder:")
autoencoder_history = autoencoder.fit(
    X_train_processed_padded,
    X_train_processed_padded,
    epochs=50,
    batch_size=12,
    shuffle=True,
    verbose=2,
    validation_data=(X_validation, X_validation),
    callbacks=[early_stopping])

ae_val_loss = np.min(autoencoder_history.history['val_loss'])
print(f"1dCNN autoencoder trained, with validation loss: {ae_val_loss}.")


(None, 52, 28)
(None, 52, 32)
(None, 26, 32)
(None, 26, 16)
(None, 13, 16)
(None, 13, 16)
(None, 26, 16)
(None, 26, 32)
(None, 52, 32)
(None, 52, 28)
Training log for 1dCNN autoencoder:
Epoch 1/50
23/23 - 2s - loss: 0.1518 - val_loss: 0.0500 - 2s/epoch - 75ms/step
Epoch 2/50
23/23 - 0s - loss: 0.0250 - val_loss: 0.0140 - 185ms/epoch - 8ms/step
Epoch 3/50
23/23 - 0s - loss: 0.0131 - val_loss: 0.0112 - 174ms/epoch - 8ms/step
Epoch 4/50
23/23 - 0s - loss: 0.0112 - val_loss: 0.0099 - 172ms/epoch - 7ms/step
Epoch 5/50
23/23 - 0s - loss: 0.0101 - val_loss: 0.0092 - 177ms/epoch - 8ms/step
Epoch 6/50
23/23 - 0s - loss: 0.0094 - val_loss: 0.0083 - 158ms/epoch - 7ms/step
Epoch 7/50
23/23 - 0s - loss: 0.0087 - val_loss: 0.0077 - 149ms/epoch - 6ms/step
Epoch 8/50
23/23 - 0s - loss: 0.0080 - val_loss: 0.0071 - 203ms/epoch - 9ms/step
Epoch 9/50
23/23 - 0s - loss: 0.0074 - val_loss: 0.0066 - 161ms/epoch - 7ms/step
Epoch 10/50
23/23 - 0s - loss: 0.0068 - val_loss: 0.0060 - 162ms/epoch - 7ms/step
Epoch

In [ ]:
from _guided import get_global_weights
PRED_MARGIN_W_LIST = [1]
from help_functions import evaluate
w_type = "global"
pos_label = 1
neg_label = 0

if w_type == "global":
    step_weights = get_global_weights(
        X,
        y_classes,
        cnnClassifier,
        n_timesteps= n_timesteps,
        n_features=n_features,
        random_state=RANDOM_STATE,
    )
elif w_type == "uniform":
    step_weights = np.ones((1, n_timesteps_padded, n_features))
elif w_type.lower() == "local":
    step_weights = "local"
else:
    raise NotImplementedError(
        "A.w_type not implemented, please choose 'local', 'global' or 'uniform'."
    )
### Evaluation metrics
for pred_margin_weight in PRED_MARGIN_W_LIST:
    print(f"The current prediction margin weight is {pred_margin_weight}.")

    lr_list = [0.001]


13/13 [==============================] - 0s 5ms/step
The current prediction margin weight is 1.


In [ ]:
step_weights

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]]])

In [ ]:
reset_seeds()
cf_model = ModifiedLatentCF(
    probability=0.5,tolerance=1e-6, max_iter=500, optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001),autoencoder = autoencoder,
    pred_margin_weight=0.7, step_weights = step_weights, random_state= RANDOM_STATE)
cf_model.fit(cnnClassifier)

y_neg = y_classes[y_classes == 0][10:15]
X_neg = X[y_classes == 0][10:15]


with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning) # ignore warning of matrix multiplication: https://stackoverflow.com/questions/29688168/mean-nanmean-and-warning-mean-of-empty-slice
    cf_embeddings, losses, weights = cf_model.transform(X_neg, y_neg) #self, x, pred_label
cf_pred_labels = cnnClassifier.predict(cf_embeddings)[:,1]# predicted probabilities of CFs
for idx in range(cf_pred_labels.shape[0]):
  if cf_pred_labels[idx] > 0.5:
    cf_pred_labels[idx] = 1
  else:
    cf_pred_labels[idx] = 0

print(f'Transformation_finished with validity_score = {validity_score(y_neg,cf_pred_labels)}')

1 samples been transformed.
5 samples been transformed, in total.
1/1 [==============================] - 0s 56ms/step
Transformation_finished with validity_score = 1.0


In [ ]:
#Proximity
def euclidean_distance(X, cf_samples):
    paired_distances = np.linalg.norm(X - cf_samples, axis=1)
    return np.mean(paired_distances)
euclidean_distance(X_neg, cf_embeddings)

0.40918086563892114

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.stats import gaussian_kde

# Reshape and standardize the data
X_reshaped = X.reshape(X.shape[0], n_timesteps_padded * n_features)
scaler = StandardScaler()
X_standardized = scaler.fit_transform(X_reshaped)

# Reduce dimensions using PCA
pca = PCA(n_components=1500)  # for example, reducing to 100 dimensions
X_pca = pca.fit_transform(X_standardized)

# Now apply KDE on the reduced data
# For example, applying KDE on the first component
#kernel = gaussian_kde(X_pca[:, 0])


ValueError: ignored

In [ ]:
normal_data.shape

(208, 52)

In [ ]:
abnormal_data[:, np.newaxis].shape

In [ ]:
for i in range(cf_embeddings.shape[2]):
  dimention = i+1

  #X_stacked_data.shape = (554, 408, 61)

  abnormal_data = X[y_classes == 1][:,:,dimention]
  normal_data = X[y_classes == 0][:,:,dimention]
  #abnormal_data.shape =normal_data.shape =  (295, 408)

  #cf_embeddings.shape = (5, 408, 61)
  counterf_data = cf_embeddings[:,:,dimention]
  #counterf_data.shape = (5, 408)

  #get the kernel for every dimention of the trained
  kernel = gaussian_kde(abnormal_data.T,bw_method=0.5)

LinAlgError: ignored

In [ ]:
from scipy.stats import gaussian_kde

kernel_x = gaussian_kde(x_eights_train.T,bw_method=None)


log_likelihood_of_xtest_eights = np.mean(kernel_x.logpdf(x_eights_test.T))
print(f"log_likelihood_of_xtest_eights = {log_likelihood_of_xtest_eights}")

log_likelihood_of_xtest_zeroes = np.mean(kernel_x.logpdf(x_zeros_test.T))
print(f"log_likelihood_of_xtest_zeroes = {log_likelihood_of_xtest_zeroes}")


log_likelihood_of_xcf = np.mean(kernel_x.logpdf(x_eights_cf.T))
print(f"log_likelihood_of_xcf = {log_likelihood_of_xcf}\n")

kernel_y = gaussian_kde(y_eights_train.T,bw_method=None)


log_likelihood_of_ytest_eights = np.mean(kernel_y.logpdf(y_eights_test.T))
print(f"log_likelihood_of_ytest_eights = {log_likelihood_of_ytest_eights}")

log_likelihood_of_ytest_zeroes = np.mean(kernel_y.logpdf(y_zeros_test.T))
print(f"log_likelihood_of_ytest_zeroes = {log_likelihood_of_ytest_zeroes}")


log_likelihood_of_ycf = np.mean(kernel_y.logpdf(y_eights_cf.T))
print(f"log_likelihood_of_ycf = {log_likelihood_of_ycf}")

In [ ]:
x_eights_train = X_train_processed_padded[y_train_classes == 1][:,:,0]
y_eights_train = X_train_processed_padded[y_train_classes == 1][:,:,1]
print(f"shape of eights_train = {x_eights_train.shape}")

x_eights_test = X_test_processed_padded[y_test_classes == 1][:,:,0]
y_eights_test = X_test_processed_padded[y_test_classes == 1][:,:,1]
print(f"shape of eights_test = {x_eights_test.shape}")

x_zeros_test = X_test_processed_padded[y_test_classes == 0][:,:,0]
y_zeros_test = X_test_processed_padded[y_test_classes == 0][:,:,1]
print(f"shape of zeros_test = {x_zeros_test.shape}")

x_eights_cf = cf_embeddings[:,:,0]
y_eights_cf = cf_embeddings[:,:,1]
print(f"shape of eights_cf = {x_eights_cf.shape}")

In [ ]:
from scipy.stats import gaussian_kde

kernel_x = gaussian_kde(x_eights_train.T,bw_method=None)


log_likelihood_of_xtest_eights = np.mean(kernel_x.logpdf(x_eights_test.T))
print(f"log_likelihood_of_xtest_eights = {log_likelihood_of_xtest_eights}")

log_likelihood_of_xtest_zeroes = np.mean(kernel_x.logpdf(x_zeros_test.T))
print(f"log_likelihood_of_xtest_zeroes = {log_likelihood_of_xtest_zeroes}")


log_likelihood_of_xcf = np.mean(kernel_x.logpdf(x_eights_cf.T))
print(f"log_likelihood_of_xcf = {log_likelihood_of_xcf}\n")

kernel_y = gaussian_kde(y_eights_train.T,bw_method=None)


log_likelihood_of_ytest_eights = np.mean(kernel_y.logpdf(y_eights_test.T))
print(f"log_likelihood_of_ytest_eights = {log_likelihood_of_ytest_eights}")

log_likelihood_of_ytest_zeroes = np.mean(kernel_y.logpdf(y_zeros_test.T))
print(f"log_likelihood_of_ytest_zeroes = {log_likelihood_of_ytest_zeroes}")


log_likelihood_of_ycf = np.mean(kernel_y.logpdf(y_eights_cf.T))
print(f"log_likelihood_of_ycf = {log_likelihood_of_ycf}")

In [ ]:
#Calculating proximity
from tensorflow.keras.losses import MeanSquaredError
total = 0
probability = 0.5
for idx in range(cf_embeddings.shape[0]):
    counterfactual = cf_embeddings[idx,np.newaxis]
    prediction = cnnClassifier.predict(counterfactual)[:, 1]
    dist = (prediction - probability)
    total +=dist
mean_mse = total /cf_embeddings.shape[0]
print(f"The Mean MSE of the data is: {mean_mse} ")